# Importing the libraries

## Function to install/Uninstall libraries

In [ ]:
import sys
import subprocess
assert sys.version_info >= (3,5)

def install_packages(pkg_name, 
                     pkg_version = '', 
                     medium = 'pip'):
    
    print(f'Installing {pkg_name} {pkg_version}...',end='')
    try:
        if len(pkg_version) == 0:
            subprocess.check_call([sys.executable,
                                   '-m',
                                   medium,
                                   'install',
                                   pkg_name])
#             !{sys.executable} -m pip install --yes <pkg_name>
#             !conda install --yes --predix {sys.prefix} <pkg_name>
        else:
            subprocess.check_call([sys.executable,
                                   '-m',
                                   medium,
                                   'install',
                                   pkg_name+'=='+pkg_version])
        print('Installed successfully.')
    except:
        print(f'Unable to install! Please install it manually.')
    
def uninstall_packages(pkg_name, 
                     medium = 'pip'):
    
    print(f'Uninstalling {pkg_name}...',end='')
    try:
        subprocess.check_call([sys.executable,
                               '-m',
                               medium,
                               'uninstall',
                               pkg_name,
                               '-y'])
#             !{sys.executable} -m pip uninstall --yes <pkg_name>
#             !conda uninstall --yes --predix {sys.prefix} <pkg_name>
        print('Uninstalled successfully.')
    except:
        print(f'Unable to uninstall! Please uninstall it manually.')

## Installing openpyxl

In [ ]:
try:
    pkg_name = 'openpyxl'
    pkg_version = ''
    import openpyxl
    assert openpyxl.__version__ >= pkg_version
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import openpyxl
    assert openpyxl.__version__ >= pkg_version
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Installing scipy

In [ ]:
try:
    pkg_name = 'scipy'
    pkg_version = ''
    import scipy
    assert scipy.__version__ >= pkg_version
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import scipy
    assert scipy.__version__ >= pkg_version
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Installing Pands

In [ ]:
try:
    pkg_name = 'pandas'
    pkg_version = ''
    import pandas as pd
    assert pd.__version__ >= pkg_version
except ModuleNotFoundError:
    install_packages(pkg_name=pkg_name)
    import pandas as pd
    assert pd.__version__ >= pkg_version
except:
    print(f'Please install {pkg_name} {pkg_version} manually.')

## Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.io
import openpyxl

# #  Class to load mat file using scipy and save to excel/database

In [ ]:
class mat_reader():
    
    def __init__(self, parent_dir):
        self.intialization(parent_dir)
        
    def intialization(self, parent_dir):
        self.count = 0
        self.parent_dir = parent_dir
        
    def _save_excel(self,
                    array, 
                    file_path):
#         array = np.where(isinstance(array, scipy.io.matlab.mio5_params.mat_struct), self._todict(array), array)
        for i in range(array.shape[0]):
            if isinstance(array[i], scipy.io.matlab.mio5_params.mat_struct):
                array[i] = self._to_np_array(array[i])
        pd.DataFrame(array).to_excel(os.path.join(self.parent_dir, file_path), 
                              sheet_name=os.path.splitext(file_path)[0],
                              header=False,
                              index=False)
        
    def convert_mat_to_excel(self,
                             d,
                             indent=0,
                             nkeys=0,
                             file_name=''):
        if nkeys>0:
            d = {k:d[k] for k in list(d.keys())[:nkeys]}
            
        if isinstance(d,dict):
            wb = openpyxl.Workbook()
            ws1 = wb.active
            ws1.title = os.path.splitext(file_name)[0]
            row_no = 0
            for key,value in d.items():
                print('\t' * indent + f'Key: {key}')
                if isinstance(value, scipy.io.matlab.mio5_params.mat_struct):
                    value = self._todict(value)
                if isinstance(value, dict):
                    row_no += 1
                    self.count += 1
                    ws1.cell(row=row_no, coloum=1).value = f'{key}'
                    ws1.cell(row=row_no, coloum=2).value = f'({len(value)}) dict'
                    ws1.cell(row=row_no, coloum=2).hyperlink = f'{key}_{self.count}.xlsx'
                    ws1.cell(row=row_no, coloum=2).style = 'Hyperlink'
                    self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')
                elif isinstance(value, np.ndarray):
                    row_no += 1
                    self.count += 1
                    ws1.cell(row=row_no, column=1).value = f'{key}'
                    ws1.cell(row=row_no, column=2).value = f'({value.shape}) ndarray'
                    ws1.cell(row=row_no, column=2).hyperlink = f'{key}_{self.count}.xlsx'
                    ws1.cell(row=row_no, column=2).style = 'Hyperlink'
                    if value.dtype.names is not None:
                        self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')
                    else:
                        self._save_excel(value, f'{key}_{self.count}.xlsx')
                else:
                    row_no += 1
                    ws1.cell(row=row_no, column=1).value = f'{key}'
                    ws1.cell(row=row_no, column=2).value = f'{value}'
            wb.save(filename = os.path.join(self.parent_dir, file_name))
        elif isinstance(d,np.ndarray) and d.dtype.names is not None:  # Note: and short-circuits by default
            for n in d.dtype.names:    # This means it's a struct, it's bit of a kludge test.
                print('\t' * indent + 'Field: ' + str(n))
                self.count += 1
                self.convert_mat_to_excel(value, indent+1, file_name = f'{key}_{self.count}.xlsx')        
                
    def loadmat(self, filename):
        '''
        this function should be called instead of direct spio.loadmat
        as it cures the problem of not properly recovering python dictionaries
        from mat files. It calls the function check keys to cure all entries
        which are still mat-objects

        from: `StackOverflow <http://stackoverflow.com/questions/7008608/scipy-io-loadmat-nested-structures-i-e-dictionaries>`_
        '''
        data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
        return self._check_keys(data)

    def _check_keys(self, dic):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in dic:
            if isinstance(dic[key], scipy.io.matlab.mio5_params.mat_struct):
                dic[key] = self._todict(dic[key])
        return dic    

    def _todict(self, matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        dic = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                dic[strg] = _todict(elem)
            else:
                dic[strg] = elem
        return dic

    def _to_np_array(self, matobj):
        '''
        A recursive function which constructs from matobjects nested arrays
        '''
        lst = []
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                lst.append(_to_np_array(elem))
            else:
                lst.append(elem)
        return np.array(lst)                  

## Loading and converting mat file into excel

In [ ]:
def convert():
    mat_file_path = r'sample.mat'
    parent_dir = os.path.splitext(mat_file_path)[0]
    os.makedirs(parent_dir, exist_ok = True)
    m_reader  = mat_reader(parent_dir)
    m_reader.convert_mat_to_excel(m_reader.loadmat(mat_file_path), file_name = f'{mat_file_path}.xlsx')
convert()